# RAG Evaluation
_Authored by: [Aymeric Roucher](https://huggingface.co/m-ric)_

This notebook demonstrates how you can evaluate your RAG (Retrieval Augmented Generation), by building a synthetic evaluation dataset and using LLM-as-a-judge to compute the accuracy of your system.

For an introduction to RAG, you can check [this other cookbook](rag_zephyr_langchain)!

RAG systems are complex: here a RAG diagram, where we noted in blue all possibilities for system enhancement:

<img src="https://huggingface.co/datasets/huggingface/cookbook-images/resolve/main/RAG_workflow.png" height="700">

Implementing any of these improvements can bring a huge performance boost; but changing anything is useless if you cannot monitor the impact of your changes on the system's performance!
So let's see how to evaluate our RAG system.

### Evaluating RAG performance

Since there are so many moving parts to tune with a big impact on performance, benchmarking the RAG system is crucial.

For our evaluation pipeline, we will need:
1. An evaluation dataset with question - answer couples (QA couples)
2. An evaluator to compute the accuracy of our system on the above evaluation dataset.

➡️ It turns out, we can use LLMs to help us all along the way!
1. The evaluation dataset will be synthetically generated by an LLM 🤖, and questions will be filtered out by other LLMs 🤖
2. An [LLM-as-a-judge](https://huggingface.co/papers/2306.05685) agent 🤖 will then perform the evaluation on this synthetic dataset.

__Let's dig into it and start building our evaluation pipeline!__ First, we install the required model dependancies.

In [1]:
# !pip install -q torch transformers langchain sentence-transformers tqdm openpyxl openai pandas datasets langchain-community ragatouille

In [ ]:
import pandas as pd
import datasets
import pickle
import os
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
import pickle

# Configure API key
dashscope_api_key = "<paste_your_api_key_here>"
dashscope_base_url = "<paste_your_base_url_here>"


pd.set_option("display.max_colwidth", None)

### Load your knowledge base

In [2]:
with open("./Data/all_splits.pkl", "rb") as f:
    docs_processed = pickle.load(f)

# 1. Build a synthetic dataset for evaluation
We first build a synthetic dataset of questions and associated contexts. The method is to get elements from our knowledge base, and ask an LLM to generate questions based on these documents.

Then we setup other LLM agents to act as quality filters for the generated QA couples: each of them will act as the filter for a specific flaw.

### 1.1. Prepare source documents

In [4]:
prompt = ChatPromptTemplate.from_messages([
    ("system", 
      """ Your task is to write a factoid question in Arabic only and an answer given a context in Arabic only.
Your factoid question should be answerable with a specific, concise piece of factual information from the context.
Your factoid question should be formulated in the same style as questions users could ask in a search engine.
This means that your factoid question MUST NOT mention something like "according to the passage" or "context".

Provide your answer as follows:

Output:::
Factoid question: (your factoid question in Arabic)
Answer: (your answer to the factoid question in Arabic)
"""),
    ("human", """Now here is the context.

Context: {context}\n
Output:::""")
])



### 1.2. Setup agents for question generation

We use [Mixtral](https://huggingface.co/mistralai/Mixtral-8x7B-Instruct-v0.1) for QA couple generation because it it has excellent performance in leaderboards such as [Chatbot Arena](https://huggingface.co/spaces/lmsys/chatbot-arena-leaderboard).

In [5]:
llm = ChatOpenAI(
    model_name="qwen-plus",  # or qwen-Turpo, qwen-plus, qwen-max,deepseek-r1, qwen3-8b
    openai_api_key=dashscope_api_key,
    openai_api_base=dashscope_base_url,
    temperature=0.4,
    # extra_body={"enable_thinking": False} 
)




Now let's generate our QA couples.
For this example, we generate only 10 QA couples and will load the rest from the Hub.

But for your specific knowledge base, given that you want to get at least ~100 test samples, and accounting for the fact that we will filter out around half of these with our critique agents later on, you should generate much more, in the >200 samples.

In [40]:
import random
from tqdm import tqdm

N_GENERATIONS = 469

print(f"Generating {N_GENERATIONS} QA couples...")

outputs = []
for sampled_context in tqdm(random.sample(docs_processed, N_GENERATIONS)):
    try:
        # Create and run the chain
        chain = prompt | llm | StrOutputParser()
        output_QA_couple = chain.invoke({"context": sampled_context.page_content})
        
        question = output_QA_couple.split("Factoid question: ")[-1].split("Answer: ")[0]
        answer = output_QA_couple.split("Answer: ")[-1]
        
        assert len(answer) < 300, "Answer is too long"
        
        outputs.append({
            "context": sampled_context.page_content,
            "question": question,
            "answer": answer,
            "source_doc": sampled_context.metadata["source"],
        })
    except Exception as e:
        print(f"Error: {e}")
        continue

Generating 469 QA couples...


  1%|          | 5/469 [00:17<26:31,  3.43s/it]

Error: Error code: 400 - {'error': {'code': 'data_inspection_failed', 'param': None, 'message': 'Input data may contain inappropriate content.', 'type': 'data_inspection_failed'}, 'id': 'chatcmpl-1cae3525-9489-9656-ae5f-f080a6d2b294', 'request_id': '1cae3525-9489-9656-ae5f-f080a6d2b294'}


  1%|▏         | 6/469 [00:20<25:40,  3.33s/it]

Error: Error code: 400 - {'error': {'code': 'data_inspection_failed', 'param': None, 'message': 'Input data may contain inappropriate content.', 'type': 'data_inspection_failed'}, 'id': 'chatcmpl-410cc838-6ca4-9e91-a777-505d36ae6b08', 'request_id': '410cc838-6ca4-9e91-a777-505d36ae6b08'}


...

Error: Error code: 400 - {'error': {'code': 'data_inspection_failed', 'param': None, 'message': 'Input data may contain inappropriate content.', 'type': 'data_inspection_failed'}, 'id': 'chatcmpl-a4dafe72-2a3b-9edf-a13c-c7e8c172ba4f', 'request_id': 'a4dafe72-2a3b-9edf-a13c-c7e8c172ba4f'}


100%|█████████▉| 468/469 [26:11<00:04,  4.25s/it]

Error: Error code: 400 - {'error': {'code': 'data_inspection_failed', 'param': None, 'message': 'Input data may contain inappropriate content.', 'type': 'data_inspection_failed'}, 'id': 'chatcmpl-8666d1b8-65e0-9546-9115-89124b48f6fb', 'request_id': '8666d1b8-65e0-9546-9115-89124b48f6fb'}


100%|██████████| 469/469 [26:14<00:00,  3.36s/it]

Error: Error code: 400 - {'error': {'code': 'data_inspection_failed', 'param': None, 'message': 'Input data may contain inappropriate content.', 'type': 'data_inspection_failed'}, 'id': 'chatcmpl-b077c03a-20ed-9128-9fd2-874e67d813b2', 'request_id': 'b077c03a-20ed-9128-9fd2-874e67d813b2'}


### 1.3. Setup critique agents

The questions generated by the previous agent can have many flaws: we should do a quality check before validating these questions.

We thus build critique agents that will rate each question on several criteria, given in [this paper](https://huggingface.co/papers/2312.10003):
- **Groundedness:** can the question be answered from the given context?
- **Relevance:** is the question relevant to users? For instance, `"What is the date when transformers 4.29.1 was released?"` is not relevant for ML practitioners.

One last failure case we've noticed is when a function is tailored for the particular setting where the question was generated, but undecipherable by itself, like `"What is the name of the function used in this guide?"`.
We also build a critique agent for this criteria:
- **Stand-alone**: is the question understandable free of any context, for someone with domain knowledge/Internet access? The opposite of this would be `What is the function used in this article?` for a question generated from a specific blog article.

We systematically score functions with all these agents, and whenever the score is too low for any one of the agents, we eliminate the question from our eval dataset.

💡 ___When asking the agents to output a score, we first ask them to produce its rationale. This will help us verify scores, but most importantly, asking it to first output rationale gives the model more tokens to think and elaborate an answer before summarizing it into a single score token.___

We now build and run these critique agents.

In [32]:
question_groundedness_critique_prompt = """
You will be given a context and a question.
Your task is to provide a 'total rating' scoring how well one can answer the given question unambiguously with the given context.
Give your answer on a scale of 1 to 5, where 1 means that the question is not answerable at all given the context, and 5 means that the question is clearly and unambiguously answerable with the context.

Provide your answer as follows:

Answer:::
Evaluation: (your rationale for the rating, as a text)
Total rating: (your rating, as a number between 1 and 5)

You MUST provide values for 'Evaluation:' and 'Total rating:' in your answer.

Now here are the question and context.

Question: {question}\n
Context: {context}\n
Answer::: """

question_relevance_critique_prompt = """
You will be given a question.
Your task is to provide a 'total rating' representing how useful this question can be useful to Muslim person .
Give your answer on a scale of 1 to 5, where 1 means that the question is not useful at all, and 5 means that the question is extremely useful.

Provide your answer as follows:

Answer:::
Evaluation: (your rationale for the rating, as a text)
Total rating: (your rating, as a number between 1 and 5)

You MUST provide values for 'Evaluation:' and 'Total rating:' in your answer.

Now here is the question.

Question: {question}\n
Answer::: """

question_standalone_critique_prompt = """
You will be given a question.
Your task is to provide a 'total rating' representing how context-independent this question is.
Give your answer on a scale of 1 to 5, where 1 means that the question depends on additional information to be understood, and 5 means that the question makes sense by itself.
For instance, if the question refers to a particular setting, like 'in the context' or 'in the document', the rating must be 1.
The questions can contain obscure technical nouns or acronyms like Gradio, Hub, Hugging Face or Space and still be a 5: it must simply be clear to an operator with access to documentation what the question is about.

For instance, "What is the name of the checkpoint from which the ViT model is imported?" should receive a 1, since there is an implicit mention of a context, thus the question is not independent from the context.

Provide your answer as follows:

Answer:::
Evaluation: (your rationale for the rating, as a text)
Total rating: (your rating, as a number between 1 and 5)

You MUST provide values for 'Evaluation:' and 'Total rating:' in your answer.

Now here is the question.

Question: {question}\n
Answer::: """

In [41]:
from langchain_core.prompts import ChatPromptTemplate

# Create prompt templates
groundedness_prompt = ChatPromptTemplate.from_template(question_groundedness_critique_prompt)
relevance_prompt = ChatPromptTemplate.from_template(question_relevance_critique_prompt)
standalone_prompt = ChatPromptTemplate.from_template(question_standalone_critique_prompt)

print("Generating critique for each QA couple...")

# Create the chains properly
chain1 = groundedness_prompt | llm | StrOutputParser()
chain2 = relevance_prompt | llm | StrOutputParser()
chain3 = standalone_prompt | llm | StrOutputParser()

for output in tqdm(outputs):      
    try:
        evaluations = {
        "groundedness": chain1.invoke({"context": output["context"], "question": output["question"]}),
        "relevance": chain2.invoke({"question": output["question"]}),
        "standalone": chain3.invoke({"question": output["question"]}),
    }
        for criterion, evaluation in evaluations.items():
            score = int(evaluation.split("Total rating: ")[-1].strip())
            eval = evaluation.split("Total rating: ")[-2].split("Evaluation: ")[1]
            output.update({
                f"{criterion}_score": score,
                f"{criterion}_eval": eval,
            })
    except Exception as e:
        continue

Generating critique for each QA couple...


  0%|          | 0/382 [00:00<?, ?it/s]

100%|██████████| 382/382 [1:50:22<00:00, 17.34s/it]   


Now let us filter out bad questions based on our critique agent scores:

In [44]:
import pandas as pd

pd.set_option("display.max_colwidth", None)

generated_questions = pd.DataFrame.from_dict(outputs)

print("Evaluation dataset before filtering:")
display(
    generated_questions[
        [
            "question",
            "answer",
            "groundedness_score",
            "relevance_score",
            "standalone_score",
        ]
    ]
)
generated_questions = generated_questions.loc[
    (generated_questions["groundedness_score"] >= 3)
    & (generated_questions["relevance_score"] >= 3)
    & (generated_questions["standalone_score"] >= 2)
]
print("============================================")
print("Final evaluation dataset:")
display(
    generated_questions[
        [
            "question",
            "answer",
            "groundedness_score",
            "relevance_score",
            "standalone_score",
        ]
    ]
)

eval_dataset = datasets.Dataset.from_pandas(
    generated_questions, split="train", preserve_index=False
)

Evaluation dataset before filtering:


,question,answer,groundedness_score,relevance_score,standalone_score
0,من خلق الملائكة حسب ما ورد في النص؟\n,الله عز وجل,NaN,NaN,NaN
1,ما هو المثل الذي استخدمه الرسول لوصف كيف سيهاجم الأعداء المسلمين؟\n,مثل الأكلة إلى قصعتها,5.0,3.0,4.0
2,ما الذي لا يعقل نسبته إلى الحجارة الصماء وفقًا للنص؟\n,القدرة على الخلق والرزق والإحياء والإماتة وإيصال النفع والضر إلى من تشاء.,5.0,5.0,1.0
3,ما هي الكلمات التي يجب أن يقولها المصلي عند القعدة حسب حديث أبوموسى الأشعري؟\n,التحيات الطيبات الصلوات لله؛ السلام عليك أيها النبي ورحمة الله وبركاته؛ السلام علينا وعلى عباد الله الصالحين؛ أشهد أن لا إله إلا الله وأشهد أن محمداً عبده ورسوله,5.0,5.0,2.0
4,ما هو الحكم الشرعي في ذبائح أهل الكتاب وفقاً للنص؟\n,الحل,5.0,5.0,1.0
...,...,...,...,...,...
377,من الذي جاء إلى النبي محمد مع عظم رميم في يده؟\n,أبو بن خلف أو العاص بن واثل,3.0,4.0,4.0
378,ماذا كان رد فعل النبي عندما ذكرت بعض نسائه الكنيسة التي رأينها بأرض الحبشة؟\n,"رفع رأسه وقال: ""أولئك إذا مات منهم الرجل الصالح بنوا على قبره مسجداً؛ ثم صوروا فيه تلك الصورة أولئك شرار الخلق عند الله""",5.0,4.0,1.0
379,كم عدد المرات التي ذكر فيها النبي أن الله سيخرجه من النار من قال لا إله إلا الله؟\n,مرة واحدة,2.0,5.0,2.0
380,ما هي الكلمات التي كان الرسول يعلّمها في التشهد كما يعلّم السورة من القرآن؟\n,"""التحيات المباركات الصلوات الطيبات لله؛ السلام عليك أيها النبي ورحمة الله وبركاته.؛ السلام علينا وعلى عباد الله الصالحين؛ أشهد أن لا إله إلا الله وأشهد أن محمداً رسول الله""",5.0,5.0,5.0


Final evaluation dataset:


,question,answer,groundedness_score,relevance_score,standalone_score
1,ما هو المثل الذي استخدمه الرسول لوصف كيف سيهاجم الأعداء المسلمين؟\n,مثل الأكلة إلى قصعتها,5.0,3.0,4.0
3,ما هي الكلمات التي يجب أن يقولها المصلي عند القعدة حسب حديث أبوموسى الأشعري؟\n,التحيات الطيبات الصلوات لله؛ السلام عليك أيها النبي ورحمة الله وبركاته؛ السلام علينا وعلى عباد الله الصالحين؛ أشهد أن لا إله إلا الله وأشهد أن محمداً عبده ورسوله,5.0,5.0,2.0
5,ما هو الحكم الشرعي إذا زوج الرجل ابنته وهي كارهة؟\n,الزواج مردود.,5.0,5.0,5.0
6,كم مرة تكون صلاة الجماعة أفضل من صلاة الفذ؟\n,سبع وعشرين درجة,5.0,4.0,5.0
7,ما هي العبادة التي جاء بها جميع الأنبياء؟\n,عبادة الله وحده,5.0,5.0,5.0
...,...,...,...,...,...
372,ما هي الحالات الثلاث التي يحل فيها دم المسلم؟\n,النفس، والثيب الزاني، والتارك لدينه المفارق للجماعة,5.0,5.0,5.0
373,ما هو المقصود بذات عرق في الحديث؟\n,المقصود بذات عرق هو الجبل الصغير الموجود فيها.,5.0,5.0,2.0
375,"من الذي روى الحديث ""أنا أغنى الشركاء عن الشرك من عمل عملا أشرك فيه معي غيري تركته وشركه""؟\n",مسلم,5.0,5.0,5.0
377,من الذي جاء إلى النبي محمد مع عظم رميم في يده؟\n,أبو بن خلف أو العاص بن واثل,3.0,4.0,4.0


Now our synthetic evaluation dataset is complete! We can evaluate different RAG systems on this evaluation dataset.

We have generated only a few QA couples here to reduce time and cost. But let's kickstart the next part by loading a pre-generated dataset: